In [1]:
# Mount Google Drive
from google.colab import drive
from pathlib import Path
import os


# Mount Drive to access dataset and model folders
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
#  Setup: imports + model/processor + device

import os, json
from pathlib import Path
import numpy as np
from PIL import Image, ImageEnhance
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification

# your model folder
MODEL_DIR = Path("/content/drive/MyDrive/ecoscan/models/vit_ecoscan_v1")

# Pick device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load processor + model (make sure these match the model you trained)
processor = AutoImageProcessor.from_pretrained(MODEL_DIR)
model = AutoModelForImageClassification.from_pretrained(MODEL_DIR).to(device)
model.eval()

print("num_labels:", model.config.num_labels)

# show human-readable class names if available
id2label = getattr(model.config, "id2label", None)
class_names = [id2label.get(str(i), id2label.get(i, str(i))) for i in range(model.config.num_labels)] if id2label else None
print("class_names:", class_names if class_names else "(no id2label found, will show numeric IDs)")



Using device: cuda
num_labels: 15
class_names: ['battery', 'brown-glass', 'cardboard', 'clothes', 'electronics', 'green-glass', 'metal_packaging', 'oil', 'organic', 'paper', 'plastic', 'shoes', 'tetrapak', 'trash', 'white-glass']


The model and processor loaded correctly and we’re using the GPU (cuda) for fast inference. The printout confirms a ViT with 12 transformer layers, hidden size 768, patch size 16×16, and a classifier head with 15 outputs (your 15 recycling classes). The model is in eval() mode, so we’re ready to run real-photo tests in the next cells.

In [15]:
# Load the saved Energy threshold (from 04)
# We will use Energy (T=1.0) and the chosen threshold to accept/reject

EVAL_DIR = MODEL_DIR / "eval"
with open(EVAL_DIR / "reject_threshold.json") as f:
    cfg = json.load(f)

thr = float(cfg["reject_threshold"])
print("Loaded threshold:", thr)
print("Config keys:", list(cfg.keys()))
print("Rule:", cfg.get("rule", "accept if energy <= threshold; else reject"))


Loaded threshold: -4.222049236297607
Config keys: ['method', 'reject_threshold', 'target_ood_reject', 'rule', 'metrics_at_threshold']
Rule: accept if energy <= threshold; else reject


We successfully loaded the saved operating point: Energy (T=1.0) with threshold ≈ −5.4664.
Rule to use from now on: accept if energy ≤ threshold; reject otherwise.
This is the 85% OOD-rejection setting we chose, so every new photo will be judged by this simple rule.

In [16]:
# Core helper: compute Energy + accept/reject

@torch.no_grad()
def logits_from_image(img: Image.Image):
    """Get logits from the model for a single PIL image."""
    enc = processor(images=img.convert("RGB"), return_tensors="pt").to(device)
    out = model(**enc)
    return out.logits.detach().cpu().numpy()[0]  # shape: (num_classes,)

def energy_from_logits_np(logits: np.ndarray, T: float = 1.0) -> float:
    """
    Energy score (T=1.0 by default).
    Lower energy = more likely in-domain.
    """
    x = logits / T
    m = np.max(x)
    lse = m + np.log(np.exp(x - m).sum())
    return float(-T * lse)

def predict_one_with_reject(img_path: str, class_names=None):
    """
    Returns a dict with: pred_class_id, pred_class_name, max_softmax, energy, rejected
    """
    img = Image.open(img_path).convert("RGB")
    logits = logits_from_image(img)
    probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
    pred_id = int(np.argmax(probs))
    pred_name = class_names[pred_id] if class_names is not None else str(pred_id)
    energy = energy_from_logits_np(logits, T=1.0)
    rejected = energy > thr  # IMPORTANT: accept if energy <= thr, reject otherwise
    return {
        "path": img_path,
        "pred_class_id": pred_id,
        "pred_class_name": pred_name,
        "max_softmax": float(probs[pred_id]),
        "energy": energy,
        "threshold": thr,
        "rejected": bool(rejected),
    }


In [19]:
# Load real photos from Drive (ID: subfolders per class, OOD: one folder)

from pathlib import Path

PHOTOS_ROOT = Path("/content/drive/MyDrive/ecoscan/data/my_photos")
ID_DIR  = PHOTOS_ROOT / "id"   # in-domain photos -> subfolders named exactly like your classes
OOD_DIR = PHOTOS_ROOT / "ood"  # out-of-domain photos

IMG_EXTS = {".jpg", ".jpeg", ".png", ".webp", ".bmp"}

def gather_id_samples(id_root, valid_class_names=None):
    """Return list of (path, expected_class_name). Expects subfolders per class."""
    items = []
    if not id_root.exists():
        print(f"Missing folder: {id_root}")
        return items

    # build a case-insensitive map to the model's canonical class names (if available)
    name_map = None
    if valid_class_names:
        name_map = {str(n).lower(): str(n) for n in valid_class_names}

    for cls_dir in sorted(p for p in id_root.iterdir() if p.is_dir()):
        folder_name = cls_dir.name
        expected = folder_name
        if name_map:
            expected = name_map.get(folder_name.lower(), folder_name)  # tolerate case differences
            if expected not in valid_class_names:
                print(f" Folder '{folder_name}' not recognized in model classes. Using as-is.")
        for p in sorted(cls_dir.iterdir()):
            if p.is_file() and p.suffix.lower() in IMG_EXTS:
                items.append((str(p), expected))
    return items

def gather_paths(folder):
    if not folder.exists():
        print(f" Missing folder: {folder}")
        return []
    return [str(p) for p in sorted(folder.iterdir())
            if p.is_file() and p.suffix.lower() in IMG_EXTS]

# Use class_names from your Celda 2 (may be None, that’s fine)
ID_SAMPLES  = gather_id_samples(ID_DIR, valid_class_names=class_names)
OOD_SAMPLES = gather_paths(OOD_DIR)

print(f"Found {len(ID_SAMPLES)} in-domain photos with expected class.")
print(f"Found {len(OOD_SAMPLES)} OOD photos.")


Found 3 in-domain photos with expected class.
Found 4 OOD photos.


In [20]:
# Run inference on all photos and print a per-image table

from pathlib import Path

results = []

# In-domain with expected class
for path, expected in ID_SAMPLES:
    r = predict_one_with_reject(path, class_names=class_names)
    r["expected_type"] = "ID"
    r["expected_class_name"] = expected
    r["decision"] = "ACCEPT" if not r["rejected"] else "REJECT"
    # outcomes
    r["is_correct_class"] = (not r["rejected"]) and (r["pred_class_name"] == expected)
    r["is_wrong_class"]   = (not r["rejected"]) and (r["pred_class_name"] != expected)
    r["is_false_reject"]  = r["rejected"]
    results.append(r)

# OOD (no expected class)
for path in OOD_SAMPLES:
    r = predict_one_with_reject(path, class_names=class_names)
    r["expected_type"] = "OOD"
    r["expected_class_name"] = None
    r["decision"] = "ACCEPT" if not r["rejected"] else "REJECT"
    r["is_correct_ood_reject"] = r["rejected"]
    r["is_false_accept"]       = not r["rejected"]
    results.append(r)

# Pretty print per image
print("file | expected_type | expected_class | pred | prob | energy | thr | decision | note")
for r in results:
    fname = Path(r["path"]).name
    if r["expected_type"] == "ID":
        note = "correct" if r["is_correct_class"] else ("wrong-class" if r["is_wrong_class"] else "false-reject")
        print(f"{fname} | ID | {r['expected_class_name']} | {r['pred_class_name']} | "
              f"{r['max_softmax']:.3f} | {r['energy']:.3f} | {r['threshold']:.3f} | {r['decision']} | {note}")
    else:
        note = "rejected" if r["is_correct_ood_reject"] else "false-accept"
        print(f"{fname} | OOD | - | {r['pred_class_name']} | "
              f"{r['max_softmax']:.3f} | {r['energy']:.3f} | {r['threshold']:.3f} | {r['decision']} | {note}")


file | expected_type | expected_class | pred | prob | energy | thr | decision | note
detergent_box_01.JPG | ID | cardboard | paper | 0.929 | -6.512 | -4.222 | ACCEPT | wrong-class
crushed_can_01.JPG | ID | metal_packaging | metal_packaging | 0.632 | -4.336 | -4.222 | ACCEPT | correct
crushed_plastic_bottle_01.JPG | ID | plastic | plastic | 0.981 | -6.945 | -4.222 | ACCEPT | correct
cat_01.JPG | OOD | - | clothes | 0.241 | -3.190 | -4.222 | REJECT | rejected
coffe_pod_01.JPG | OOD | - | metal_packaging | 0.348 | -3.444 | -4.222 | REJECT | rejected
plant_01.JPG | OOD | - | cardboard | 0.461 | -3.999 | -4.222 | REJECT | rejected
thermomix_01.JPG | OOD | - | plastic | 0.414 | -3.544 | -4.222 | REJECT | rejected


Using the Energy threshold ≈ −4.22, the demo works as intended:

In-domain (3 photos): all ACCEPTED.

Plastic bottle: correct.

Crushed can: correct (now passes the gate).

Detergent box: ACCEPTED but wrong class (paper vs expected cardboard).

OOD (4 photos): all REJECTED (cat, coffee pod, plant, thermomix).

So the reject-unknowns gate is solid, and class predictions are mostly right on your real photos, with one reasonable confusion (paper vs cardboard).

In [21]:
# Summary counts for the demo
id_total = sum(1 for r in results if r["expected_type"]=="ID")
id_correct = sum(1 for r in results if r["expected_type"]=="ID" and r["is_correct_class"])
id_wrong   = sum(1 for r in results if r["expected_type"]=="ID" and r["is_wrong_class"])
id_rej     = sum(1 for r in results if r["expected_type"]=="ID" and r["is_false_reject"])

ood_total = sum(1 for r in results if r["expected_type"]=="OOD")
ood_rej   = sum(1 for r in results if r["expected_type"]=="OOD" and r["is_correct_ood_reject"])
ood_acc   = sum(1 for r in results if r["expected_type"]=="OOD" and r["is_false_accept"])

print("== Demo summary ==")
print(f"ID accepted & correct: {id_correct}/{id_total} ({(id_correct/max(1,id_total))*100:.1f}%)")
print(f"ID accepted but wrong-class: {id_wrong}/{id_total} ({(id_wrong/max(1,id_total))*100:.1f}%)")
print(f"ID rejected (false-reject): {id_rej}/{id_total} ({(id_rej/max(1,id_total))*100:.1f}%)")
print(f"OOD rejected: {ood_rej}/{ood_total} ({(ood_rej/max(1,ood_total))*100:.1f}%)")
print(f"OOD accepted (false-accept): {ood_acc}/{ood_total} ({(ood_acc/max(1,ood_total))*100:.1f}%)")


== Demo summary ==
ID accepted & correct: 2/3 (66.7%)
ID accepted but wrong-class: 1/3 (33.3%)
ID rejected (false-reject): 0/3 (0.0%)
OOD rejected: 4/4 (100.0%)
OOD accepted (false-accept): 0/4 (0.0%)


In [23]:
# Robustness check (fixed): brightness & rotation using PIL images directly
from PIL import Image, ImageEnhance
from pathlib import Path

TEST_ID  = ID_SAMPLES[0][0] if ID_SAMPLES else None
TEST_OOD = OOD_SAMPLES[0]    if OOD_SAMPLES else None

def check(img_path, tag):
    if not img_path or not Path(img_path).exists():
        print(f"No valid {tag} image path.");
        return

    base = Image.open(img_path).convert("RGB")

    def infer_from_pil(img_obj):
        # Use the helper that accepts PIL images
        logits = logits_from_image(img_obj)
        probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
        pred_id = int(np.argmax(probs))
        pred_name = class_names[pred_id] if class_names is not None else str(pred_id)
        energy = energy_from_logits_np(logits, T=1.0)
        rejected = energy > thr  # accept if energy <= thr
        return pred_name, float(probs[pred_id]), float(energy), bool(rejected)

    print(f"\n== {tag} ==")
    variants = [
        ("base", base),
        ("brightness x0.7", ImageEnhance.Brightness(base).enhance(0.7)),
        ("brightness x1.3", ImageEnhance.Brightness(base).enhance(1.3)),
        ("+15°", base.rotate(15, expand=True)),
        ("-15°", base.rotate(-15, expand=True)),
    ]
    for name, img in variants:
        pred, prob, energy, rejected = infer_from_pil(img)
        print(f"{Path(img_path).name} [{name}] -> pred={pred}, "
              f"prob={prob:.3f}, energy={energy:.3f}, thr={thr:.3f}, rejected={rejected}")

check(TEST_ID, "ID perturbations")
check(TEST_OOD, "OOD perturbations")



== ID perturbations ==
detergent_box_01.JPG [base] -> pred=paper, prob=0.929, energy=-6.512, thr=-4.222, rejected=False
detergent_box_01.JPG [brightness x0.7] -> pred=paper, prob=0.835, energy=-6.090, thr=-4.222, rejected=False
detergent_box_01.JPG [brightness x1.3] -> pred=paper, prob=0.946, energy=-6.851, thr=-4.222, rejected=False
detergent_box_01.JPG [+15°] -> pred=paper, prob=0.971, energy=-6.937, thr=-4.222, rejected=False
detergent_box_01.JPG [-15°] -> pred=paper, prob=0.988, energy=-7.597, thr=-4.222, rejected=False

== OOD perturbations ==
cat_01.JPG [base] -> pred=clothes, prob=0.241, energy=-3.190, thr=-4.222, rejected=True
cat_01.JPG [brightness x0.7] -> pred=clothes, prob=0.386, energy=-3.307, thr=-4.222, rejected=True
cat_01.JPG [brightness x1.3] -> pred=shoes, prob=0.165, energy=-3.180, thr=-4.222, rejected=True
cat_01.JPG [+15°] -> pred=cardboard, prob=0.467, energy=-3.440, thr=-4.222, rejected=True
cat_01.JPG [-15°] -> pred=cardboard, prob=0.646, energy=-3.784, thr=-4

The accept/reject gate is robust to small lighting/angle changes; one class confusion (paper vs cardboard) persists, which is fine to mention as a known limitation.

In [24]:
# Finalize: save demo results to Drive (JSON + CSV)
from pathlib import Path
import json, csv

OUT = MODEL_DIR / "eval"
OUT.mkdir(parents=True, exist_ok=True)

# JSON
with open(OUT / "inference_demo_results.json", "w") as f:
    json.dump(results, f, indent=2)

# CSV
def note_for(r):
    if r["expected_type"] == "ID":
        return "correct" if r["is_correct_class"] else ("wrong-class" if r["is_wrong_class"] else "false-reject")
    else:
        return "rejected" if r["is_correct_ood_reject"] else "false-accept"

csv_path = OUT / "inference_demo_results.csv"
with open(csv_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["path","expected_type","expected_class","pred","prob","energy","threshold","decision","rejected","note"])
    for r in results:
        w.writerow([
            r["path"], r["expected_type"], r.get("expected_class_name"),
            r["pred_class_name"], f"{r['max_softmax']:.5f}",
            f"{r['energy']:.5f}", f"{r['threshold']:.5f}",
            r["decision"], r["rejected"], note_for(r)
        ])

print("Saved:", OUT / "inference_demo_results.json")
print("Saved:", OUT / "inference_demo_results.csv")
print("DONE: 05 notebook ready ✅")


Saved: /content/drive/MyDrive/ecoscan/models/vit_ecoscan_v1/eval/inference_demo_results.json
Saved: /content/drive/MyDrive/ecoscan/models/vit_ecoscan_v1/eval/inference_demo_results.csv
DONE: 05 notebook ready ✅


Demo summary. Using the saved Energy threshold (≈ −4.22, 75% OOD target), all in-domain photos were accepted (two with the correct class; one reasonable paper vs cardboard confusion) and all OOD photos were rejected. We also tried small lighting/rotation changes: decisions stayed stable. We saved the per-image results to eval/inference_demo_results.json and .csv for reporting and reproducibility.